In [114]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np

# Loading CSV file

In [115]:
df_anime = pd.read_csv(r'data\myAnimelist-No_Hentai 2.csv')
df_rating = pd.read_csv(r'data\rating.csv')

In [116]:
df_anime.rename(columns={"animeID":'anime_id'},inplace=True)

# Create a new database

In [136]:
# Database Path object for easier calling
db_path = r"db\anime.db"

In [137]:
# Create a new database file

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    # connect to database file
    try:
        conn = sqlite3.connect(db_file)
        print(f'sqlite3 version: {sqlite3.version}')
        print('Created Database')
    # print error if connection can not be established
    except Error as error:
        print(error)
        print('Could not create Database')
    # close connection
    finally:
        if conn:
            conn.close()
    

# run the above function (save database file to path)
if __name__ == '__main__':
    create_connection(db_path)

sqlite3 version: 2.6.0
Created Database


In [138]:
# # Create a new database that resides in the memory (RAM)

# def create_connection():
#     """ create a database connection to a database that resides
#         in the memory
#     """
#     conn = None;
#     # connect to memory
#     try:
#         conn = sqlite3.connect(':memory:')
#         print(f'sqlite3 version: {sqlite3.version}')
#         print('Created Database')
#     # print error if connection can not be established
#     except Error as error:
#         print(error)
#         print('Could not create Database')
#     # close connection
#     finally:
#         if conn:
#             conn.close()
    

# # run the above function
# if __name__ == '__main__':
#     create_connection()


# Create Tables

continue at https://www.sqlitetutorial.net/sqlite-python/create-tables/

In [139]:
# function for establish connection to SQLite3 for editing database

def establish_connection(db_file):
    """ 
    creates a database connection to a SQLite database file
    
    :returns: Connection object or None
    """
    conn = None
    # connect to database file
    try:
        conn = sqlite3.connect(db_file)
        print(f'sqlite3 version: {sqlite3.version}')
        print('Created Database')
    # print error if connection can not be established
    except Error as error:
        print(error)
        print('Could not create Database')
    
    return conn

In [140]:
# function for creating table in database

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        # make object cursor from connect
        c = conn.cursor()
        # from cursor use execute to create table
        c.execute(create_table_sql)
    except Error as error:
        print(error)

In [141]:
#

def main():
    
    sql_create_anime_table = """ CREATE TABLE IF NOT EXISTS Anime (
                                        anime_id integer PRIMARY KEY NOT NULL,
                                        name text NOT NULL,
                                        type text,
                                        source text,
                                        episodes integer,
                                        duration datetime,
                                        genre text,
                                        rating text,
                                        score real,
                                        scored_by integer,
                                        rank integer,
                                        popularity integer,
                                        members integer,
                                        favorites integer,
                                        FOREIGN KEY (anime_id) REFERENCES rating (anime_id)
                                    ); """

    sql_create_rating_table = """CREATE TABLE IF NOT EXISTS Rating (
                                    anime_id integer PRIMARY KEY NOT NULL,
                                    user_id integer NOT NULL,
                                    rating integer,
                                    FOREIGN KEY (anime_id) REFERENCES anime (anime_id)
                                    
                                );"""

    # create a database connection
    conn = establish_connection(db_path)

    # create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_anime_table)

        # create tasks table
        create_table(conn, sql_create_rating_table)
    else:
        print("Error! cannot create the database connection.")

    
if __name__ == '__main__':
    main()

sqlite3 version: 2.6.0
Created Database


In [142]:
display(df_anime.head(5))
display(df_rating.head(5))

,anime_id,name,type,source,episodes,duration,genre,rating,score,scored_by,rank,popularity,members,favorites
0,1,Cowboy Bebop,TV,Original,26,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460
1,5,Cowboy Bebop: Tengoku no Tobira,Movie,Original,1,1:55:00,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",R - 17+ (violence & profanity),8.41,120243,164,449,197791,776
2,6,Trigun,TV,Manga,26,0:24:00,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432
3,7,Witch Hunter Robin,TV,Original,26,0:25:00,"['Action', 'Magic', 'Police', 'Supernatural', ...",PG-13 - Teens 13 or older,7.33,32837,2371,1171,79397,537
4,8,Bouken Ou Beet,TV,Manga,52,0:23:00,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",PG - Children,7.03,4894,3544,3704,11708,14


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


# Populating Tables from csv files

In [143]:
# importing data from dataframe 'anime' to table anime
df_anime.to_sql('Anime',sqlite3.connect(db_path), if_exists='replace')

#import data from dataframe 'rating' to table rating 
df_rating.to_sql('Rating',sqlite3.connect(db_path), if_exists='replace')

In [144]:
# close connection to database

def close_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        conn.close()
        print('Connection successfully closed')
    except Error as error:
        print(error)
        print('Could not close connection')
# run the above function (save database file to path)
if __name__ == '__main__':
    close_connection(db_path)

Connection successfully closed


# Querying data with conditions and saving to Pandas Dataframe

In [151]:
conn = sqlite3.connect(db_path)

In [155]:

df_score_greater_then_8 = pd.read_sql_query("""SELECT * 
                                               from Anime 
                                               where score>8 
                                               order by score desc;""", conn)
df_score_greater_then_8

,index,anime_id,name,type,source,episodes,duration,genre,rating,score,scored_by,rank,popularity,members,favorites
0,5551,38781,Violence Voyager,Movie,Original,1,1:23:00,"['Adventure', 'Comedy', 'Horror']",R - 17+ (violence & profanity),10.00,1,13544,14765,65,0
1,5318,37149,Xing You Ji: Fengbao Famila,ONA,Original,1,1:10:00,"['Sci-Fi', 'Adventure', 'Fantasy']",PG-13 - Teens 13 or older,9.33,3,13633,12651,171,0
2,5539,38570,Tatakae! Dokan-kun: Robolympic-hen,TV,Original,13,0:05:00,"['Comedy', 'Mecha']",G - All Ages,9.25,4,13288,15264,31,0
3,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
4,2580,9253,Steins;Gate,TV,Visual novel,24,0:24:00,"['Thriller', 'Sci-Fi']",PG-13 - Teens 13 or older,9.14,633590,3,7,1139182,104173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,2616,9513,Beelzebub,TV,Manga,60,0:24:00,"['Action', 'Comedy', 'Demons', 'Supernatural',...",PG-13 - Teens 13 or older,8.01,148483,554,215,332769,4097
437,3971,24921,Aoki Hagane no Arpeggio: Ars Nova Cadenza,Movie,Manga,1,1:45:00,"['Action', 'Sci-Fi', 'Seinen']",PG-13 - Teens 13 or older,8.01,8034,552,2831,20938,105
438,4075,27775,Plastic Memories,TV,Original,13,0:24:00,"['Sci-Fi', 'Drama', 'Romance']",PG-13 - Teens 13 or older,8.01,197098,557,155,396716,6153
439,4297,30415,High☆Speed!: Free! Starting Days,Movie,Light novel,1,1:50:00,"['Slice of Life', 'Comedy', 'Sports', 'Drama',...",PG-13 - Teens 13 or older,8.01,17893,555,1823,44425,202
